In [57]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
import tensorflow as tf
import csv

In [2]:
url = 'https://www.cs.cmu.edu/~spok/grimmtmp/'

# Create a directory if needed
dir_name = 'stories'
if not os.path.exists(dir_name):
    os.mkdir(dir_name)
    
def maybe_download(filename):
  """Download a file if not present"""
  print('Downloading file: ', dir_name+ os.sep+filename)
    
  if not os.path.exists(dir_name+os.sep+filename):
    filename, _ = urlretrieve(url + filename, dir_name+os.sep+filename)
  else:
    print('File ',filename, ' already exists.')
  
  return filename

num_files = 100
filenames = [format(i, '03d')+'.txt' for i in range(1,101)]

for fn in filenames:
    maybe_download(fn)

In [58]:
for i in range(len(filenames)):
    file_exists = os.path.isfile(os.path.join(dir_name,filenames[i]))
    assert file_exists
print('%d files found.'%len(filenames))

100 files found.


In [59]:
def read_data(filename):
  
  with open(filename) as f:
    data = tf.compat.as_str(f.read())
    # make all the words lower case
    data = data.lower()
    data = list(data)
  return data

documents = []
global documents
for i in range(num_files):    
    print('\nProcessing file %s'%os.path.join(dir_name,filenames[i]))
    chars = read_data(os.path.join(dir_name,filenames[i]))
    
    # Break the data into bigrams
    two_grams = [''.join(chars[ch_i:ch_i+2]) for ch_i in range(0,len(chars)-2,2)]
    # Create a list of lists with bigrams
    documents.append(two_grams)
    print('Data size (Characters) (Document %d) %d' %(i,len(two_grams)))
    print('Sample string (Document %d) %s'%(i,two_grams[:50]))


Processing file stories\001.txt
Data size (Characters) (Document 0) 3667
Sample string (Document 0) ['in', ' o', 'ld', 'en', ' t', 'im', 'es', ' w', 'he', 'n ', 'wi', 'sh', 'in', 'g ', 'st', 'il', 'l ', 'he', 'lp', 'ed', ' o', 'ne', ', ', 'th', 'er', 'e ', 'li', 've', 'd ', 'a ', 'ki', 'ng', '\nw', 'ho', 'se', ' d', 'au', 'gh', 'te', 'rs', ' w', 'er', 'e ', 'al', 'l ', 'be', 'au', 'ti', 'fu', 'l,']

Processing file stories\002.txt
Data size (Characters) (Document 1) 4928
Sample string (Document 1) ['ha', 'rd', ' b', 'y ', 'a ', 'gr', 'ea', 't ', 'fo', 're', 'st', ' d', 'we', 'lt', ' a', ' w', 'oo', 'd-', 'cu', 'tt', 'er', ' w', 'it', 'h ', 'hi', 's ', 'wi', 'fe', ', ', 'wh', 'o ', 'ha', 'd ', 'an', '\no', 'nl', 'y ', 'ch', 'il', 'd,', ' a', ' l', 'it', 'tl', 'e ', 'gi', 'rl', ' t', 'hr', 'ee']

Processing file stories\003.txt
Data size (Characters) (Document 2) 9745
Sample string (Document 2) ['a ', 'ce', 'rt', 'ai', 'n ', 'fa', 'th', 'er', ' h', 'ad', ' t', 'wo', ' s', 'on', 's,', ' 

In [60]:
#Building the Dictionaries (Bigrams)
#Builds the following. To understand each of these elements, let us also assume the text "I like to go to school"
#
#dictionary: maps a string word to an ID (e.g. {I:0, like:1, to:2, go:3, school:4})
#reverse_dictionary: maps an ID to a string word (e.g. {0:I, 1:like, 2:to, 3:go, 4:school}
#count: List of list of (word, frequency) elements (e.g. [(I,1),(like,1),(to,2),(go,1),(school,1)]
#data : Contain the string of text we read, where string words are replaced with word IDs (e.g. [0, 1, 2, 3, 2, 4])
#It also introduces an additional special token UNK to denote rare words to are too rare to make use of.

In [61]:
def build_dataset(documents):
    chars = []
    # This is going to be a list of lists
    # Where the outer list denote each document
    # and the inner lists denote words in a given document
    data_list = []
  
    for d in documents:
        chars.extend(d)
    print('%d Characters found.'%len(chars))
    count = []
    # Get the bigram sorted by their frequency (Highest comes first)
    count.extend(collections.Counter(chars).most_common())
    
    # Create an ID for each bigram by giving the current length of the dictionary
    # And adding that item to the dictionary
    # Start with 'UNK' that is assigned to too rare words
    dictionary = dict({'UNK':0})
    for char, c in count:
        # Only add a bigram to dictionary if its frequency is more than 10
        if c > 10:
            dictionary[char] = len(dictionary)    
    
    unk_count = 0
    # Traverse through all the text we have
    # to replace each string word with the ID of the word
    for d in documents:
        data = list()
        for char in d:
            # If word is in the dictionary use the word ID,
            # else use the ID of the special token "UNK"
            if char in dictionary:
                index = dictionary[char]        
            else:
                index = dictionary['UNK']
                unk_count += 1
            data.append(index)
            
        data_list.append(data)
        
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return data_list, count, dictionary, reverse_dictionary

global data_list, count, dictionary, reverse_dictionary,vocabulary_size

# Print some statistics about data
data_list, count, dictionary, reverse_dictionary = build_dataset(documents)
print('Most common words (+UNK)', count[:5])
print('Least common words (+UNK)', count[-15:])
print('Sample data', data_list[0][:10])
print('Sample data', data_list[1][:10])
print('Vocabulary: ',len(dictionary))
vocabulary_size = len(dictionary)
del documents  # To reduce memory.

449177 Characters found.
Most common words (+UNK) [('e ', 15229), ('he', 15164), (' t', 13443), ('th', 13076), ('d ', 10687)]
Least common words (+UNK) [('bj', 1), ('ii', 1), ('i?', 1), ('z ', 1), ('c.', 1), ('"k', 1), ('pw', 1), ('f?', 1), (' z', 1), ('xq', 1), ('nm', 1), ('m?', 1), ('\t"', 1), ('\tw', 1), ('tz', 1)]
Sample data [15, 28, 86, 23, 3, 95, 74, 11, 2, 16]
Sample data [22, 156, 25, 37, 82, 185, 43, 9, 90, 19]
Vocabulary:  544


In [62]:
#Generating Batches of Data
#The following object generates a batch of data which will be used to train the LSTM. 
#More specifically the generator breaks a given sequence of words into batch_size segments. 
#We also maintain a cursor for each segment. So whenever we create a batch of data, 
#we sample one item from each segment and update the cursor of each segment.

In [63]:
class DataGeneratorOHE(object):
    
    def __init__(self,text,batch_size,num_unroll):
        # Text where a bigram is denoted by its ID
        self._text = text
        # Number of bigrams in the text
        self._text_size = len(self._text)
        # Number of datapoints in a batch of data
        self._batch_size = batch_size
        # Num unroll is the number of steps we unroll the RNN in a single training step
        # This relates to the truncated backpropagation we discuss in Chapter 6 text
        self._num_unroll = num_unroll
        # We break the text in to several segments and the batch of data is sampled by
        # sampling a single item from a single segment
        self._segments = self._text_size//self._batch_size
        self._cursor = [offset * self._segments for offset in range(self._batch_size)]
        
    def next_batch(self):
        '''
        Generates a single batch of data
        '''
        # Train inputs (one-hot-encoded) and train outputs (one-hot-encoded)
        batch_data = np.zeros((self._batch_size,vocabulary_size),dtype=np.float32)
        batch_labels = np.zeros((self._batch_size,vocabulary_size),dtype=np.float32)
        
        # Fill in the batch datapoint by datapoint
        for b in range(self._batch_size):
            # If the cursor of a given segment exceeds the segment length
            # we reset the cursor back to the beginning of that segment
            if self._cursor[b]+1>=self._text_size:
                self._cursor[b] = b * self._segments
            
            # Add the text at the cursor as the input
            batch_data[b,self._text[self._cursor[b]]] = 1.0
            # Add the preceding bigram as the label to be predicted
            batch_labels[b,self._text[self._cursor[b]+1]]= 1.0                       
            # Update the cursor
            self._cursor[b] = (self._cursor[b]+1)%self._text_size
                    
        return batch_data,batch_labels
        
    def unroll_batches(self):
        '''
        This produces a list of num_unroll batches
        as required by a single step of training of the RNN
        '''
        unroll_data,unroll_labels = [],[]
        for ui in range(self._num_unroll):
            data, labels = self.next_batch()            
            unroll_data.append(data)
            unroll_labels.append(labels)
        
        return unroll_data, unroll_labels
    
    def reset_indices(self):
        '''
        Used to reset all the cursors if needed
        '''
        self._cursor = [offset * self._segments for offset in range(self._batch_size)]
        
# Running a tiny set to see if things are correct
dg = DataGeneratorOHE(data_list[0][25:50],5,5)
u_data, u_labels = dg.unroll_batches()

# Iterate through each data batch in the unrolled set of batches
for ui,(dat,lbl) in enumerate(zip(u_data,u_labels)):   
    print('\n\nUnrolled index %d'%ui)
    dat_ind = np.argmax(dat,axis=1)
    lbl_ind = np.argmax(lbl,axis=1)
    print('\tInputs:')
    for sing_dat in dat_ind:
        print('\t%s (%d)'%(reverse_dictionary[sing_dat],sing_dat),end=", ")
    print('\n\tOutput:')
    for sing_lbl in lbl_ind:        
        print('\t%s (%d)'%(reverse_dictionary[sing_lbl],sing_lbl),end=", ")



Unrolled index 0
	Inputs:
	e  (1), 	ki (131), 	 d (48), 	 w (11), 	be (70), 
	Output:
	li (98), 	ng (33), 	au (195), 	er (14), 	au (195), 

Unrolled index 1
	Inputs:
	li (98), 	ng (33), 	au (195), 	er (14), 	au (195), 
	Output:
	ve (41), 	
w (169), 	gh (106), 	e  (1), 	ti (112), 

Unrolled index 2
	Inputs:
	ve (41), 	
w (169), 	gh (106), 	e  (1), 	ti (112), 
	Output:
	d  (5), 	ho (62), 	te (61), 	al (84), 	fu (228), 

Unrolled index 3
	Inputs:
	d  (5), 	ho (62), 	te (61), 	al (84), 	fu (228), 
	Output:
	a  (82), 	se (58), 	rs (137), 	l  (57), 	l, (257), 

Unrolled index 4
	Inputs:
	a  (82), 	se (58), 	rs (137), 	l  (57), 	be (70), 
	Output:
	ki (131), 	 d (48), 	 w (11), 	be (70), 	au (195), 

In [64]:
# Number of neurons in the hidden state variables
num_nodes = 128

# Number of data points in a batch we process
batch_size = 64

# Number of time steps we unroll for during optimization
num_unrollings = 50

dropout = 0.0 # We use dropout

# Use this in the CSV filename when saving
# when using dropout
filename_extension = ''
if dropout>0.0:
    filename_extension = '_dropout'
    
filename_to_save = 'lstm'+filename_extension+'.csv' # use to save perplexity values

In [14]:
""" 
tf.reset_default_graph() 

# Training Input data.
train_inputs, train_labels = [],[]

# Defining unrolled training inputs
for ui in range(num_unrollings):
    train_inputs.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size],name='train_inputs_%d'%ui))
    train_labels.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size], name = 'train_labels_%d'%ui))

# Validation data placeholders
valid_inputs = tf.placeholder(tf.float32, shape=[1,vocabulary_size],name='valid_inputs')
valid_labels = tf.placeholder(tf.float32, shape=[1,vocabulary_size], name = 'valid_labels')
# Text generation: batch 1, no unrolling.
test_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size], name = 'test_input')
"""

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [65]:
# Training Input data.
train_inputs, train_labels = [], []

# Defining unrolled training inputs
for ui in range(num_unrollings):
    train_inputs.append(tf.keras.Input(shape=(vocabulary_size,), name='train_inputs_%d'%ui))
    train_labels.append(tf.keras.Input(shape=(vocabulary_size,), name='train_labels_%d'%ui))

# Validation data placeholders
valid_inputs = tf.keras.Input(shape=(vocabulary_size,), name='valid_inputs')
valid_labels = tf.keras.Input(shape=(vocabulary_size,), name='valid_labels')

# Text generation: batch 1, no unrolling.
test_input = tf.keras.Input(shape=(vocabulary_size,), name='test_input')

In [18]:
"""
# Input gate (i_t) - How much memory to write to cell state
# Connects the current input to the input gate
ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))
# Connects the previous hidden state to the input gate
im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], stddev=0.02))
# Bias of the input gate
ib = tf.Variable(tf.random_uniform([1, num_nodes],-0.02, 0.02))

# Forget gate (f_t) - How much memory to discard from cell state
# Connects the current input to the forget gate
fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))
# Connects the previous hidden state to the forget gate
fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], stddev=0.02))
# Bias of the forget gate
fb = tf.Variable(tf.random_uniform([1, num_nodes],-0.02, 0.02))

# Candidate value (c~_t) - Used to compute the current cell state
# Connects the current input to the candidate
cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))
# Connects the previous hidden state to the candidate
cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], stddev=0.02))
# Bias of the candidate
cb = tf.Variable(tf.random_uniform([1, num_nodes],-0.02,0.02))

# Output gate (o_t) - How much memory to output from the cell state
# Connects the current input to the output gate
ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))
# Connects the previous hidden state to the output gate
om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], stddev=0.02))
# Bias of the output gate
ob = tf.Variable(tf.random_uniform([1, num_nodes],-0.02,0.02))


# Softmax Classifier weights and biases.
w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], stddev=0.02))
b = tf.Variable(tf.random_uniform([vocabulary_size],-0.02,0.02))

# Variables saving state across unrollings.
# Hidden state
saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False, name='train_hidden')
# Cell state
saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False, name='train_cell')

# Same variables for validation phase
saved_valid_output = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='valid_hidden')
saved_valid_state = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='valid_cell')

# Same variables for testing phase
saved_test_output = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='test_hidden')
saved_test_state = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='test_cell')
"""

"\n# Input gate (i_t) - How much memory to write to cell state\n# Connects the current input to the input gate\nix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))\n# Connects the previous hidden state to the input gate\nim = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], stddev=0.02))\n# Bias of the input gate\nib = tf.Variable(tf.random_uniform([1, num_nodes],-0.02, 0.02))\n\n# Forget gate (f_t) - How much memory to discard from cell state\n# Connects the current input to the forget gate\nfx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))\n# Connects the previous hidden state to the forget gate\nfm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], stddev=0.02))\n# Bias of the forget gate\nfb = tf.Variable(tf.random_uniform([1, num_nodes],-0.02, 0.02))\n\n# Candidate value (c~_t) - Used to compute the current cell state\n# Connects the current input to the candidate\ncx = tf.Variable(tf.truncated_normal([vocab

In [66]:

# Input gate (i_t) - How much memory to write to cell state
# Connects the current input to the input gate
ix = tf.Variable(tf.random.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))
# Connects the previous hidden state to the input gate
im = tf.Variable(tf.random.truncated_normal([num_nodes, num_nodes], stddev=0.02))
# Bias of the input gate
ib = tf.Variable(tf.random.uniform([1, num_nodes],-0.02, 0.02))

# Forget gate (f_t) - How much memory to discard from cell state
# Connects the current input to the forget gate
fx = tf.Variable(tf.random.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))
# Connects the previous hidden state to the forget gate
fm = tf.Variable(tf.random.truncated_normal([num_nodes, num_nodes], stddev=0.02))
# Bias of the forget gate
fb = tf.Variable(tf.random.uniform([1, num_nodes],-0.02, 0.02))

# Candidate value (c~_t) - Used to compute the current cell state
# Connects the current input to the candidate
cx = tf.Variable(tf.random.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))
# Connects the previous hidden state to the candidate
cm = tf.Variable(tf.random.truncated_normal([num_nodes, num_nodes], stddev=0.02))
# Bias of the candidate
cb = tf.Variable(tf.random.uniform([1, num_nodes],-0.02,0.02))

# Output gate (o_t) - How much memory to output from the cell state
# Connects the current input to the output gate
ox = tf.Variable(tf.random.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))
# Connects the previous hidden state to the output gate
om = tf.Variable(tf.random.truncated_normal([num_nodes, num_nodes], stddev=0.02))
# Bias of the output gate
ob = tf.Variable(tf.random.uniform([1, num_nodes],-0.02,0.02))


# Softmax Classifier weights and biases.
w = tf.Variable(tf.random.truncated_normal([num_nodes, vocabulary_size], stddev=0.02))
b = tf.Variable(tf.random.uniform([vocabulary_size],-0.02,0.02))

# Variables saving state across unrollings.
# Hidden state
saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False, name='train_hidden')
# Cell state
saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False, name='train_cell')

# Same variables for validation phase
saved_valid_output = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='valid_hidden')
saved_valid_state = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='valid_cell')

# Same variables for testing phase
saved_test_output = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='test_hidden')
saved_test_state = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='test_cell')


In [34]:
# Definition of the cell computation.
#def lstm_cell(i, o, state):
#    """Create an LSTM cell"""
#    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
#    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
#    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
#    state = forget_gate * state + input_gate * tf.tanh(update)
#    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
#    return output_gate * tf.tanh(state), state


class LSTMCellLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(LSTMCellLayer, self).__init__()
        self.ix = ix
        self.im = im
        self.ib = ib
        self.fx = fx
        self.fm = fm
        self.fb = fb
        self.cx = cx
        self.cm = cm
        self.cb = cb
        self.ox = ox
        self.om = om
        self.ob = ob
        self.dropout_layer = tf.keras.layers.Dropout(rate=dropout)

    def call(self, inputs):
        i, o, state = inputs
        input_gate = tf.sigmoid(tf.matmul(i, self.ix) + tf.matmul(o, self.im) + self.ib)
        forget_gate = tf.sigmoid(tf.matmul(i, self.fx) + tf.matmul(o, self.fm) + self.fb)
        update = tf.matmul(i, self.cx) + tf.matmul(o, self.cm) + self.cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, self.ox) + tf.matmul(o, self.om) + self.ob)
        output = output_gate * tf.tanh(state)
        output = self.dropout_layer(output)
        return output, state

lstm_cell = LSTMCellLayer() 

In [46]:
## All new, thanks Blackbox
class MatMulLayer(tf.keras.layers.Layer):
    def __init__(self, weights):
        super(MatMulLayer, self).__init__()
        self.weights_var = self.add_weight(name='weights', shape=weights.shape, initializer=tf.keras.initializers.Constant(weights))

    def call(self, inputs):
        return tf.matmul(inputs, self.weights_var)

class XwPlusBLayer(tf.keras.layers.Layer):
    def __init__(self, weights, biases):
        super(XwPlusBLayer, self).__init__()
        self.weights_var = self.add_weight(name='weights', shape=weights.shape, initializer=tf.keras.initializers.Constant(weights))
        self.biases_var = self.add_weight(name='biases', shape=biases.shape, initializer=tf.keras.initializers.Constant(biases))

    def call(self, inputs):
        return tf.nn.xw_plus_b(inputs, self.weights_var, self.biases_var)
        
class ConcatLayer(tf.keras.layers.Layer):
    def __init__(self, axis=0):
        super(ConcatLayer, self).__init__()
        self.axis = axis

    def call(self, inputs):
        return tf.concat(inputs, axis=self.axis)

In [47]:
# =========================================================
#Training related inference logic

# Keeps the calculated state outputs in all the unrollings
# Used to calculate loss
outputs = list()

# These two python variables are iteratively updated
# at each step of unrolling
output = saved_output
state = saved_state

matmul_layer = MatMulLayer(w)
xw_plus_b_layer = XwPlusBLayer(w, b)
concat_layer = ConcatLayer(axis=0)

# Compute the hidden state (output) and cell state (state)
# recursively for all the steps in unrolling
for i in train_inputs:
    inputs = (i, output, state)
    output, state = lstm_cell(inputs)
    outputs.append(output)

# calculate the score values
logits = matmul_layer(concat_layer(outputs)) + b
    
# Compute predictions.
train_prediction = tf.nn.softmax(logits)

# Compute training perplexity
train_perplexity_without_exp = tf.reduce_sum(tf.concat(train_labels,0)*-tf.log(tf.concat(train_prediction,0)+1e-10))/(num_unrollings*batch_size)

# ========================================================================
# Validation phase related inference logic

# Compute the LSTM cell output for validation data
valid_output, valid_state = lstm_cell((valid_inputs, saved_valid_output, saved_valid_state))
# Compute the logits
valid_logits = xw_plus_b_layer(valid_output)

# Make sure that the state variables are updated
# before moving on to the next iteration of generation
with tf.control_dependencies([saved_valid_output.assign(valid_output),
                            saved_valid_state.assign(valid_state)]):
    valid_prediction = tf.nn.softmax(valid_logits)

# Compute validation perplexity
valid_perplexity_without_exp = tf.reduce_sum(valid_labels*-tf.log(valid_prediction+1e-10))

# ========================================================================
# Testing phase related inference logic

# Compute the LSTM cell output for testing data
test_output, test_state = lstm_cell((test_input, saved_test_output, saved_test_state))

# Make sure that the state variables are updated
# before moving on to the next iteration of generation
with tf.control_dependencies([saved_test_output.assign(test_output),
                            saved_test_state.assign(test_state)]):
    test_prediction = tf.nn.softmax(xw_plus_b_layer(test_output))

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [68]:
class MatMulLayer(tf.keras.layers.Layer):
    def __init__(self, weights):
        super(MatMulLayer, self).__init__()
        self.weights_var = self.add_weight(name='weights', shape=weights.shape, initializer=tf.keras.initializers.Constant(weights))

    def call(self, inputs):
        return tf.matmul(inputs, self.weights_var)

class XwPlusBLayer(tf.keras.layers.Layer):
    def __init__(self, weights, biases):
        super(XwPlusBLayer, self).__init__()
        self.weights_var = self.add_weight(name='weights', shape=weights.shape, initializer=tf.keras.initializers.Constant(weights))
        self.biases_var = self.add_weight(name='biases', shape=biases.shape, initializer=tf.keras.initializers.Constant(biases))

    def call(self, inputs):
        return tf.matmul(inputs, self.weights_var) + self.biases_var

class LSTMCellLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(LSTMCellLayer, self).__init__()
        self.ix = self.add_weight(name='ix', shape=ix.shape, initializer=tf.keras.initializers.Constant(ix))
        self.im = self.add_weight(name='im', shape=im.shape, initializer=tf.keras.initializers.Constant(im))
        self.ib = self.add_weight(name='ib', shape=ib.shape, initializer=tf.keras.initializers.Constant(ib))
        self.fx = self.add_weight(name='fx', shape=fx.shape, initializer=tf.keras.initializers.Constant(fx))
        self.fm = self.add_weight(name='fm', shape=fm.shape, initializer=tf.keras.initializers.Constant(fm))
        self.fb = self.add_weight(name='fb', shape=fb.shape, initializer=tf.keras.initializers.Constant(fb))
        self.cx = self.add_weight(name='cx', shape=cx.shape, initializer=tf.keras.initializers.Constant(cx))
        self.cm = self.add_weight(name='cm', shape=cm.shape, initializer=tf.keras.initializers.Constant(cm))
        self.cb = self.add_weight(name='cb', shape=cb.shape, initializer=tf.keras.initializers.Constant(cb))
        self.ox = self.add_weight(name='ox', shape=ox.shape, initializer=tf.keras.initializers.Constant(ox))
        self.om = self.add_weight(name='om', shape=om.shape, initializer=tf.keras.initializers.Constant(om))
        self.ob = self.add_weight(name='ob', shape=ob.shape, initializer=tf.keras.initializers.Constant(ob))
        self.dropout_layer = tf.keras.layers.Dropout(rate=dropout)

    def call(self, inputs):
        i, o, state = inputs
        input_gate = tf.sigmoid(tf.matmul(i, self.ix) + tf.matmul(o, self.im) + self.ib)
        forget_gate = tf.sigmoid(tf.matmul(i, self.fx) + tf.matmul(o, self.fm) + self.fb)
        update = tf.matmul(i, self.cx) + tf.matmul(o, self.cm) + self.cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, self.ox) + tf.matmul(o, self.om) + self.ob)
        output = output_gate * tf.tanh(state)
        output = self.dropout_layer(output)
        return output, state

lstm_cell = LSTMCellLayer()

# =========================================================
#Training related inference logic

# Keeps the calculated state outputs in all the unrollings
# Used to calculate loss
outputs = list()

# These two python variables are iteratively updated
# at each step of unrolling
output = saved_output
state = saved_state

matmul_layer = MatMulLayer(w)
xw_plus_b_layer = XwPlusBLayer(w, b)

# Compute the hidden state (output) and cell state (state)
# recursively for all the steps in unrolling
for i in train_inputs:
    inputs = (i, output, state)
    output, state = lstm_cell(inputs)
    outputs.append(output)

# calculate the score values
logits = xw_plus_b_layer(tf.concat(axis=0, values=outputs))
    
# Compute predictions.
train_prediction = tf.nn.softmax(logits)

# Compute training perplexity
train_perplexity

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [29]:
# Before calcualting the training loss,
# save the hidden state and the cell state to
# their respective TensorFlow variables
with tf.control_dependencies([saved_output.assign(output),
                            saved_state.assign(state)]):

    # Calculate the training loss by
    # concatenating the results from all the unrolled time steps
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=tf.concat(axis=0, values=train_labels)))

AttributeError: module 'tensorflow._api.v2.nn' has no attribute 'softmax_cross_entropy_with_logits_v2'

In [30]:
# learning rate decay
gstep = tf.Variable(0,trainable=False,name='global_step')

# Running this operation will cause the value of gstep
# to increase, while in turn reducing the learning rate
inc_gstep = tf.assign(gstep, gstep+1)

# Decays learning rate everytime the gstep increases
tf_learning_rate = tf.train.exponential_decay(0.001,gstep,decay_steps=1, decay_rate=0.5)

# Adam Optimizer. And gradient clipping.
optimizer = tf.train.AdamOptimizer(tf_learning_rate)
gradients, v = zip(*optimizer.compute_gradients(loss))
# Clipping gradients
gradients, _ = tf.clip_by_global_norm(gradients, 5.0)

optimizer = optimizer.apply_gradients(
    zip(gradients, v))

AttributeError: module 'tensorflow' has no attribute 'assign'

In [31]:
# Reset train state
reset_train_state = tf.group(tf.assign(saved_state, tf.zeros([batch_size, num_nodes])),
                          tf.assign(saved_output, tf.zeros([batch_size, num_nodes])))

# Reset valid state
reset_valid_state = tf.group(tf.assign(saved_valid_state, tf.zeros([1, num_nodes])),
                          tf.assign(saved_valid_output, tf.zeros([1, num_nodes])))

# Reset test state
reset_test_state = tf.group(
    saved_test_output.assign(tf.random_normal([1, num_nodes],stddev=0.05)),
    saved_test_state.assign(tf.random_normal([1, num_nodes],stddev=0.05)))

AttributeError: module 'tensorflow' has no attribute 'assign'

In [32]:
# Some hyperparameters needed for the training process

num_steps = 26
steps_per_document = 100
valid_summary = 1
train_doc_count = 100
docs_per_step = 10


# Capture the behavior of train perplexity over time
train_perplexity_ot = []
valid_perplexity_ot = []

session = tf.InteractiveSession()

# Initializing variables
tf.global_variables_initializer().run()
print('Initialized Global Variables ')

average_loss = 0 # Calculates the average loss ever few steps

# We use the first 10 documents that has 
# more than 10*steps_per_document bigrams for creating the validation dataset

# Identify the first 10 documents following the above condition
long_doc_ids = []
for di in range(num_files):
  if len(data_list[di])>10*steps_per_document:
    long_doc_ids.append(di)
  if len(long_doc_ids)==10:
    break
    
# Generating validation data
data_gens = []
valid_gens = []
for fi in range(num_files):
  # Get all the bigrams if the document id is not in the validation document ids
  if fi not in long_doc_ids:
    data_gens.append(DataGeneratorOHE(data_list[fi],batch_size,num_unrollings))
  # if the document is in the validation doc ids, only get up to the 
  # last steps_per_document bigrams and use the last steps_per_document bigrams as validation data
  else:
    data_gens.append(DataGeneratorOHE(data_list[fi][:-steps_per_document],batch_size,num_unrollings))
    # Defining the validation data generator
    valid_gens.append(DataGeneratorOHE(data_list[fi][-steps_per_document:],1,1))


feed_dict = {}
for step in range(num_steps):
    
    print('Training (Step: %d)'%step,end=' ')
    for di in np.random.permutation(train_doc_count)[:docs_per_step]:            
        doc_perplexity = 0
        for doc_step_id in range(steps_per_document):
            
            # Get a set of unrolled batches
            u_data, u_labels = data_gens[di].unroll_batches()
            
            # Populate the feed dict by using each of the data batches
            # present in the unrolled data
            for ui,(dat,lbl) in enumerate(zip(u_data,u_labels)):            
                feed_dict[train_inputs[ui]] = dat
                feed_dict[train_labels[ui]] = lbl
            
            # Running the TensorFlow operations
            _, l, step_perplexity = session.run([optimizer, loss, train_perplexity_without_exp], 
                                                       feed_dict=feed_dict)
            # Update doc_perpelxity variable
            doc_perplexity += step_perplexity 
            
            # Update the average_loss variable
            average_loss += step_perplexity 
            
        # Show the printing progress <train_doc_id_1>.<train_doc_id_2>. ...
        print('(%d).'%di,end='') 
    
        # resetting hidden state after processing a single document
        # It's still questionable if this adds value in terms of learning
        # One one hand it's intuitive to reset the state when learning a new document
        # On the other hand this approach creates a bias for the state to be zero
        # We encourage the reader to investigate further the effect of resetting the state
        #session.run(reset_train_state) # resetting hidden state for each document
    print('')
    
    # Generate new samples
    if (step+1) % valid_summary == 0:
      
      # Compute average loss
      average_loss = average_loss / (valid_summary*docs_per_step*steps_per_document)
      
      # Print losses
      print('Average loss at step %d: %f' % (step+1, average_loss))
      print('\tPerplexity at step %d: %f' %(step+1, np.exp(average_loss)))
      train_perplexity_ot.append(np.exp(average_loss))
        
      average_loss = 0 # reset loss
      valid_loss = 0 # reset loss
        
      # calculate valid perplexity
      for v_doc_id in range(10):
          # Remember we process things as bigrams
          # So need to divide by 2
          for v_step in range(steps_per_document//2):
            uvalid_data,uvalid_labels = valid_gens[v_doc_id].unroll_batches()        

            # Run validation phase related TensorFlow operations       
            v_perp = session.run(
                valid_perplexity_without_exp,
                feed_dict = {valid_inputs:uvalid_data[0],valid_labels: uvalid_labels[0]}
            )

            valid_loss += v_perp
            
          session.run(reset_valid_state)
      
          # Reset validation data generator cursor
          valid_gens[v_doc_id].reset_indices()      
    
      print()
      v_perplexity = np.exp(valid_loss/(steps_per_document*10.0//2))
      print("Valid Perplexity: %.2f\n"%v_perplexity)
      valid_perplexity_ot.append(v_perplexity)
          
      decay_learning_rate(session, v_perplexity)
        
      # Generating new text ...
      # We will be generating one segment having 500 bigrams
      # Feel free to generate several segments by changing
      # the value of segments_to_generate
      print('Generated Text after epoch %d ... '%step)  
      segments_to_generate = 1
      chars_in_segment = 500
    
      for _ in range(segments_to_generate):
        print('======================== New text Segment ==========================')
        
        # Start with a random word
        test_word = np.zeros((1,vocabulary_size),dtype=np.float32)
        rand_doc = data_list[np.random.randint(0,num_files)]
        test_word[0,rand_doc[np.random.randint(0,len(rand_doc))]] = 1.0
        print("\t",reverse_dictionary[np.argmax(test_word[0])],end='')
        
        # Generating words within a segment by feeding in the previous prediction
        # as the current input in a recursive manner
        for _ in range(chars_in_segment):    
          sample_pred = session.run(test_prediction, feed_dict = {test_input:test_word})  
          next_ind = sample(sample_pred.ravel())
          test_word = np.zeros((1,vocabulary_size),dtype=np.float32)
          test_word[0,next_ind] = 1.0
          print(reverse_dictionary[next_ind],end='')
        print("")
        
        # Reset train state
        session.run(reset_test_state)
        print('====================================================================')
      print("")

session.close()

# Write training and validation perplexities to a csv file
with open(filename_to_save, 'wt') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(train_perplexity_ot)
    writer.writerow(valid_perplexity_ot)

AttributeError: module 'tensorflow' has no attribute 'InteractiveSession'